# Домашнее задание № 2. Мешок слов

In [1]:
#импортируем всего да побольше
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

from razdel import tokenize
from nltk.corpus import stopwords
import numpy as np
import string

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [2]:
data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [3]:
#функция для того, чтобы из разделовского токенизатора вытаскивать только текст
razdel_tokenizer = lambda doc: [t.text for t in list(tokenize(doc))]

In [7]:
def vectorize():
    train, test = train_test_split(data, test_size=0.1, shuffle=True)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    y = train.toxic.values
    y_test = test.toxic.values
    vectorizer = CountVectorizer()
    razdel_vectorizer = CountVectorizer(tokenizer = razdel_tokenizer)
    return vectorizer, razdel_vectorizer, y_test, y, train, test

In [8]:
def KNeighbors(vectorizer, y_test, y, train, test):
    X = vectorizer.fit_transform(train.comment)
    X_test = vectorizer.transform(test.comment) 
    clf = KNeighborsClassifier(n_neighbors=10, metric='cosine')
    clf.fit(X, y)
    preds = clf.predict(X_test)
    return precision_recall_fscore_support(y_test, preds, average='weighted')
    

In [9]:
x=1
while x<=4:
    print ('эксперимент ', x)
    print('precision, recall, fscore: ')
    vectorizer, razdel_vectorizer, y_test, y, train, test = vectorize()
    print('default: ')
    precision,recall,fscore,support=KNeighbors(vectorizer, y_test, y, train, test )
    print(precision,recall,fscore)
    print('razdel: ')
    precision1,recall1,fscore1,support1 =KNeighbors(razdel_vectorizer, y_test, y, train, test )
    print(precision1,recall1,fscore1)
    print('\n\n')
    x+=1
    
    

эксперимент  1
precision, recall, fscore: 
default: 
0.7472435797885316 0.7343966712898752 0.6892376141712125
razdel: 
0.6897835525126489 0.7011095700416089 0.646813561097794



эксперимент  2
precision, recall, fscore: 
default: 
0.7582802055186355 0.7350901525658807 0.6891483404054874
razdel: 
0.7230558576264168 0.7128987517337032 0.6592177283534774



эксперимент  3
precision, recall, fscore: 
default: 
0.7550057401824901 0.7454923717059639 0.7113552400376719
razdel: 
0.7004836163990117 0.7066574202496533 0.659898558948988



эксперимент  4
precision, recall, fscore: 
default: 
0.7599255510928751 0.7510402219140083 0.7081045686482951
razdel: 
0.7190701308049154 0.7295423023578363 0.6858254116453102





В целом, 4 эксперимента показывают не очень большую разницу между двумя токенайзерами, в которой дефолтный токенизатор показывает себя лучше. Однако, судя по незначительной разнице в метриках то в одну, то в другую сторону, эти результаты практически равнозначны

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [10]:
Image(url="https://miro.medium.com/max/1200/1*V9ac4hLVyms79jl65Ym_Bw.jpeg",
     width=600, height=600)

In [11]:
#заводим массив с данными из таблицы
array = [[1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [3, 0 , 1, 1, 0, 0], [1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1]]

In [12]:
table = pd.DataFrame(array, columns = ['я', 'ты', 'и', 'только', 'не', 'он'], index = ['я и ты','ты и я','я, я и только я', 'только не я', 'он'])
table

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [13]:
#нормализуем таблицу
norm_table = table.apply(lambda x: x/ x.sum(), axis = 1)
norm_table

,я,ты,и,только,не,он
я и ты,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
ты и я,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
"я, я и только я",0.600000,0.000000,0.200000,0.200000,0.000000,0.0
только не я,0.333333,0.000000,0.000000,0.333333,0.333333,0.0
он,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [14]:
# количество документов
N = len(table)
frame = {}
# применим формулу к каждой колонке
for col in table.columns:
    df = len(table[col][table[col]!=0])
    series = norm_table[col].apply(lambda x: x*np.log(N/df))
    frame[col] = series
result = pd.DataFrame(frame)
result

,я,ты,и,только,не,он
я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [15]:
train, test = train_test_split(data, test_size=0.1, shuffle=True, random_state = 32)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
y = train.toxic.values
y_test = test.toxic.values


In [16]:
vectorizer = CountVectorizer(max_features=7000, min_df= 8, max_df=0.6, ngram_range=(1, 2), tokenizer= razdel_tokenizer, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = LogisticRegression(C=0.1,solver='lbfgs', class_weight='balanced')
clf.fit(X, y)
preds1 = clf.predict(X_test)
probas1 = clf.predict_proba(X_test)
f1_score(y_test, preds1)

0.7620751341681573

In [17]:
vectorizer = TfidfVectorizer(max_features=5000,min_df=5, max_df=0.6, ngram_range=(1, 2),tokenizer=razdel_tokenizer)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = MultinomialNB(fit_prior=False, alpha=1.2)
clf.fit(X, y)
preds2 = clf.predict(X_test)
probas2 = clf.predict_proba(X_test)
f1_score(y_test, preds2)

0.7527527527527527

In [18]:
def find_toxic(probas):
    probas = [p[1] for p in probas]
    test['probas'] = probas
    res_df = test.sort_values(by = 'probas', ascending = False)[:10]
    res_df = res_df.reset_index(drop=True)
    for i in range(10):
        print('toxic:', res_df.loc[i].toxic)
        print('comment: ', res_df.loc[i].comment)
    return res_df

In [19]:
print('CountVectorizer + LogisticRegression: ')
clf1 = find_toxic(probas1)
print('TfidfVectorizer + MultinomialNB: ')
clf2 = find_toxic(probas2)


CountVectorizer + LogisticRegression: 
toxic: 1.0
comment:  АЛЛО БЛЯДЬ АБУ ТЫ Ч ОХУЕЛ? ВЫГОНЯЙ НАХУЙ СВОЕГО ПОДЗАЛУПНОГО ДРУЖКА ВАРЛАМОВА НАХУЙ ИЗ Б ЭТА ХУЙНЯ ЗАЕБАЛА В 5 ТРЕДОВ БЛЯДЬ НА ГЛАВНОЙ ВИСЕТЬ, БАНЫЙ ТВОЙ РОТ ДЫРЯВЫЙ БЛЯДЬ, ПРОДАЖНАЯ МРАЗЬ!

toxic: 1.0
comment:  Какие блять передергивания? Ты дебил блять зашел на шок-доску и удивляешься что над тобой издеваются. Тут нет твоих друзей, рачье тупорылое, тут тебя все ненавидят. Как же печет от таких необучаемых ебланов. Ты ковбой, твою жену ебут где-то нахуй, а дети гибнут на Украине. Понял, быдло ты ебаное?

toxic: 1.0
comment:  БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. НЕПРОБИВАЕМАЯ ХОХЛИНА, СУКА. Какие-же хохлы дененераты, пиздец просто.

toxic: 1.0
comment:  А нахуй ты тут персональный чатик устроил, дегенерат? Или ты сейчас каждому работяге, который зайдет в твой обоссаный тред будешь бежать доказывать в чем он не прав и как нужно сделать шоб было всё нормально ? Проблема травлятредов была озвучена ещё задолго до того, ка

In [20]:
clf1.merge(clf2, how = 'inner', on ='comment')

,comment,toxic_x,probas_x,toxic_y,probas_y
0,АЛЛО БЛЯДЬ АБУ ТЫ Ч ОХУЕЛ? ВЫГОНЯЙ НАХУЙ СВОЕГ...,1.0,0.999889,1.0,0.996844
1,Какие блять передергивания? Ты дебил блять заш...,1.0,0.998683,1.0,0.997152
2,"БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. ...",1.0,0.998444,1.0,0.998899
3,"Блядь абу нахуй ссылай этих дегенератов в фаг,...",1.0,0.993369,1.0,0.993956
4,"предлагаю пидорнуть хохлов, хохлы не имеют отн...",1.0,0.988709,1.0,0.986386


Из 10 текстов сопало 5 -- мне кажется, это неплохой результат. Причем все совпавшие тексты являются токсичными,и из всех результатов классификатора не токичным оказался только один комментарий: "Делай людям добро - и тебе всегда будет больно и стыдно. Унижай, бей, обманывай - и будешь уважаемым человеком, тебе будут кланяться и звать по имени-отчеству. Что тебе больше нравится - выбирай сам. Выбрал боль и стыд - спасибо, дорогой. Мне меньше достанется." Возможно, из-за слов "Унижай, бей, обманывай - и будешь уважаемым человеком" 

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

# Logistic Regression

In [21]:
train, test = train_test_split(data, test_size=0.1, shuffle=True, random_state = 27)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
y = train.toxic.values
y_test = test.toxic.values
vectorizer = CountVectorizer(max_features=7000, min_df= 8, max_df=0.6, ngram_range=(1, 2), tokenizer= razdel_tokenizer, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = LogisticRegression(C=0.1,solver='lbfgs', class_weight='balanced')
clf.fit(X, y)
importance = clf.coef_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[0][:-6:-1]:
    print(words[i])

хохлы
хохлов
тебе
дебил
нахуй


# DecisionTree

In [22]:
vectorizer = CountVectorizer(max_features=7000, min_df= 8, max_df=0.6, ngram_range=(1, 2), tokenizer= lambda doc: [token for token in razdel_tokenizer(doc) if token not in string.punctuation], stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
importance = clf.feature_importances_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[:-6:-1]:
    print(words[i])

тебе
хохлы
очень
хохлов
нахуй


# Naive Bayes

In [23]:
vectorizer = TfidfVectorizer(max_features=5000,min_df=5, max_df=0.6, ngram_range=(1, 2),tokenizer=lambda doc: [token for token in razdel_tokenizer(doc) if token not in string.punctuation and token.isalpha()],  stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = MultinomialNB(fit_prior=False, alpha=1.2)
clf.fit(X, y)
importance = clf.coef_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[0][:-6:-1]:
    print(words[i])

это
тебе
просто
хохлы
хохлов


In [24]:
# я не знаю, являются ли "это" и "тебе" стоп-словами, если да, то:
stop_words=stopwords.words('russian')
stop_words.extend(['это','тебе'])
vectorizer = TfidfVectorizer(max_features=5000,min_df=5, max_df=0.6, ngram_range=(1, 2),tokenizer=lambda doc: [token for token in razdel_tokenizer(doc) if token not in string.punctuation and token.isalpha()],  stop_words=stop_words)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = MultinomialNB(fit_prior=False, alpha=1.2)
clf.fit(X, y)
importance = clf.coef_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[0][:-6:-1]:
    print(words[i])

просто
хохлы
хохлов
почему
нахуй


# Random Forest

In [25]:
vectorizer = CountVectorizer(max_features=7000, min_df= 8, max_df=0.6, ngram_range=(1, 2), tokenizer= lambda doc: [token for token in razdel_tokenizer(doc) if token not in string.punctuation], stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = RandomForestClassifier(n_estimators=100, max_depth=20, )
clf.fit(X, y)
importance = clf.feature_importances_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[:-6:-1]:
    print(words[i])

хохлы
нахуй
тебе
хохлов
блядь


В целом, основными фичами считаются слова "хохлы", "хохлов", "нахуй" для всех классификаторов.
Можно убирать слова в духе "почему", но я не уверена, что это корректно. Вполне возможно, что большая часть токсик-предложений начинаются именно с почему. И, как мне кажется, это все же не стоп-слово